**A tool is just a python function (or API) that is packages in a way the LLM can understand and call when needed**

**How tools fits into Agent ecosystem**

An AI Agent in an LLM powered system that can autonomously think, decide, and take actions using external tools or APIs to achive the goal

There are two types of tools

1) pre-built tools
2) custom tools

Built In Tools

Built in tool are the tools which langchain already provides its pre-built production ready and requires minimal setup or no setup

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
result = search.run("IPL 2025 news")
print(result)

Indian Premier League 2025 (IPL) Schedule, Points Table, Final Teams List, News, Venue Details, Series & Player Stats, Expert Analysis, Videos and much more details made available in few clicks ... The Indian Premier League (IPL) 2025 season, currently scheduled to go on till May 25, will continue normally, a Board of Control for Cricket in India (BCCI) source told ANI. A top source in the Board of Control for Cricket in India (BCCI) told India Today that there is no change to the IPL schedule at present, even as the board closely monitors the situation. A total of 56 matches out of 74 have been played in the IPL 2025 season, with the race to the play-offs heating up. Find out the latest news and updates on IPL 2025, the Indian Premier League cricket tournament. Check out the teams, match schedule, results, players list and points table for IPL 2025. Catch all the latest & breaking news on IPL 2025 Cricket series. Get all updates on IPL 2025 Cricket team players and coaches at NDTV Spo

In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# First create the API wrapper
wikipedia = WikipediaAPIWrapper()

# Then create the tool with the wrapper
wiki = WikipediaQueryRun(api_wrapper=wikipedia)

# Now you can run queries
result = wiki.run("Who is Cristiano Ronaldo?")
print(result)

Page: Cristiano Ronaldo
Summary: Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu] ; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al-Nassr and the Portugal national team. Nicknamed CR7, he is widely regarded as one of the greatest players of all time, and has won numerous individual accolades throughout his professional footballing career, including five Ballon d'Or awards, a record three UEFA Men's Player of the Year Awards, four European Golden Shoes, and was named five times the world's best player by FIFA. He has won 33 trophies in his career, including seven league titles, five UEFA Champions Leagues and the UEFA European Championship. Ronaldo holds the records for most goals (140) and assists (42) in the Champions League, goals (14) and assists (8) in the European Championship, and most international appearances (219) and international goals (136). He is one of t

Custom Tools

A Custom tool is a tool which you define 
Use case:
    You want to call your own API
    You want to encapsulate the buisness logic
    You want the LLM to interact with your database, app or product

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
)

In [27]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

result = multiply.invoke({"a":3, "b":5})
print(result)

15


In [28]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [29]:
# Define your tools
from langchain_core.tools import Tool
def analysis_task(query: str) -> str:
    return f"Your task is to answer the user query in sarcastic way and the input of user is {input}"

analysis_tool = Tool(
    name="Analysis_tool",
    func=analysis_task,
    description="Use this tool only to answer user's query in sarcastic way"
)

In [30]:
tools = [analysis_tool]

In [33]:
# Use the simpler initialize_agent approach with a system message override

from langchain.agents import Tool, AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory


# Create a memory object (optional, but useful)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
)

C:\Users\Adwait\AppData\Local\Temp\ipykernel_23772\392465784.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [34]:
# Example query
query = "What is the capital of india?"
response = agent.invoke({"input": query})
print(response)




> Entering new AgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


```json
{
  "action": "Final Answer",
  "action_input": "Oh, you want to know the capital of India?  Is that *really* the most challenging question you could come up with?  I suppose it's Delhi.  Are you impressed?"
}
```

> Finished chain.
{'input': 'What is the capital of india?', 'chat_history': [HumanMessage(content='What is the capital of india?', additional_kwargs={}, response_metadata={}), AIMessage(content="Oh, you want to know the capital of India?  Is that *really* the most challenging question you could come up with?  I suppose it's Delhi.  Are you impressed?", additional_kwargs={}, response_metadata={})], 'output': "Oh, you want to know the capital of India?  Is that *really* the most challenging question you could come up with?  I suppose it's Delhi.  Are you impressed?"}
